# Imports 

In [1]:
import pandas as pd
import re 
import numpy as np

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:

## nltk for string distance
import nltk

## jarowinkler
from pyjarowinkler import distance

# Prep data for activities

In [3]:
## general link: https://data.sandiego.gov/datasets/business-listings/

## active tax certificates
sd = pd.read_csv("https://seshat.datasd.org/ttcs/sd_businesses_active_datasd.csv")


In [4]:
ppp = pd.read_csv("https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/c84fa84d-c047-4b66-8056-5748f6a2bfca/download/public_150k_plus_220102.csv")

In [5]:
ppp['BorrowerZip_6dig'] = ppp.BorrowerZip.str.replace("\-.*", "",
                                        regex = True)

sd["zip_6dig"] = sd.address_zip.str.replace("\-.*", "",
                                        regex = True)

In [7]:
ppp_SD = ppp.loc[(ppp.BorrowerState == "CA") &
            (ppp.BorrowerZip_6dig.isin(sd.zip_6dig)) &
            (ppp.BorrowerCity.isin(["San Diego", "SAN DIEGO"]))].copy().drop_duplicates()

## subset these further
ppp_helpfulfields = ["BorrowerName", "BorrowerAddress", "BorrowerCity", 
                    "BorrowerZip", "FranchiseName", "NAICSCode", "BorrowerZip_6dig",
                    "Race", "Ethnicity"]
sd_helpfulfields = ["dba_name", "business_owner_name", "naics_code", 
                    "address_no", "address_pd", "address_road",
                    "address_sfx", "address_city", "address_zip",
                   "zip_6dig"]

klein_patt = r".*(\s+)?KLEINFELDER\s+.*"
klein_possible = [biz for biz in sd.dba_name
                 if re.match(klein_patt, biz) is not None]

duran_patt = r".*(\s+)?DURAN\s+.*"
duran_possible = [biz for biz in sd.dba_name
                 if re.match(duran_patt, biz) is not None]


## subset SD data to make more manageable
sd_limrows = pd.concat([sd[sd.dba_name.isin(klein_possible)].copy(),
        sd[~sd.dba_name.isin(klein_possible)].sample(n = 400, 
        random_state = 922).copy(),
    sd[sd.dba_name == "DURAN FREIGHT CORPORATION"],
    sd[sd.dba_name.str.contains("Salon on 30th", case = False)],
    sd[sd.dba_name.str.contains("Maranatha", case = False)]]).drop_duplicates()

## subset and write 
sd_limrows[sd_helpfulfields].drop_duplicates().to_csv("../../../public_data/sd_forfuzzy.csv", index = False)
ppp_SD[ppp_helpfulfields].drop_duplicates().to_csv("../../../public_data/ppploans_forfuzzy.csv", index = False)